# ChatGPT Capability Evaluation
This notebook evaluates the capabilities of the ChatGPT language model with respect to the task `sum_list`.
The goal is to evaluate to what cardinality the language model is capable of performing the operation `op_split` on a given list of single-digit integers.

## Task Definition
The task `sum_list` defines the following operations:
- `op_sum`: calculates the sum of a given list
- `op_split`: splits the list in two sublists of equal size
- `op_merge`: merges two lists into a single one

The scoring on `op_split` is handled by an execution operation, meaning the scoring is done by code execution.

## Define Evaluation
The evaluation requires a complete graph controller as well as a ChatGPT language model instance.

In [17]:
import getpass
import os

api_key = os.environ.get('api_key')
api_key = getpass.getpass('api_key') if api_key is None or api_key == '' else api_key
os.environ['api_key'] = api_key

In [18]:
from pure_graph_of_thoughts.language_model import ChatGPT, GPTModel
from pure_graph_of_thoughts.controller import CompleteGraphController
import logging

logging.basicConfig(level=logging.INFO)


def create_chat_gpt(model: GPTModel) -> ChatGPT:
    """
    Creates a new ChatGPT instance for a given model to use.
    :param model: model to use
    :return: ChatGPT instance
    """
    return ChatGPT(api_key=api_key, model=model)

## Prepare Storing and Loading

In [19]:
from pure_graph_of_thoughts.language_model.chatgpt.gpt_usage import GPTUsage
import json
from pure_graph_of_thoughts.api.schema import JsonSchemaEncoder
from auto_graph_of_thoughts.baseline.model import BaselineResultSummary

results_directory = '../../../artifacts/results/chatgpt_capability_evaluation/sum_list/op_split'


def get_model_dir_name(evaluated_model: GPTModel):
    """
    Gets the directory name for a given model.
    :param evaluated_model: evaluated model
    :return: the directory name of the model
    """
    return evaluated_model.id.replace('.', '').replace('-', '')


def get_results_file_name(evaluated_model: GPTModel, evaluated_operation_name: str, evaluated_cardinality: int) -> str:
    """
    Gets the file name for the results of a given operation with a given cardinality.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation
    :param evaluated_cardinality: evaluated cardinality
    :return: results file name
    """
    evaluated_model_name = get_model_dir_name(evaluated_model)
    return f'{results_directory}/{evaluated_model_name}/{evaluated_operation_name}/{evaluated_operation_name}_{str(evaluated_cardinality).zfill(2)}.json'


def store_result(evaluated_model: GPTModel, evaluated_operation_name: str, evaluated_cardinality: int,
                 result: BaselineResultSummary) -> None:
    """
    Stores a given baseline result summary as a JSON file.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation
    :param evaluated_cardinality: evaluated cardinality
    :param result: result to store
    """
    file_name = get_results_file_name(evaluated_model, evaluated_operation_name, evaluated_cardinality)
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(result, f, cls=JsonSchemaEncoder, ensure_ascii=False, indent=2)


def load_result(evaluated_model: GPTModel, evaluated_operation_name: str,
                evaluated_cardinality: int) -> BaselineResultSummary:
    """
    Loads a baseline result summary from a JSON file.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation
    :param evaluated_cardinality: evaluated cardinality
    :return: loaded baseline result summary
    """
    file_name = get_results_file_name(evaluated_model, evaluated_operation_name, evaluated_cardinality)
    with open(file_name, 'r', encoding='utf-8') as f:
        return BaselineResultSummary.from_dict(json.load(f))


def get_usage_file_name(evaluated_model: GPTModel, evaluated_operation_name: str) -> str:
    """
    Gets the file name of the model usage for a given evaluated operation.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation
    :return: usage file name
    """
    evaluated_model_name = get_model_dir_name(evaluated_model)
    return f'{results_directory}/{evaluated_model_name}/{evaluated_operation_name}/usage_{evaluated_operation_name}.json'


def store_usage(evaluated_model: GPTModel, evaluated_operation_name: str, usage: GPTUsage) -> None:
    """
    Stores the usage data of a GPT model.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation
    :param usage: usage
    """
    file_name = get_usage_file_name(evaluated_model, evaluated_operation_name)
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(usage, f, cls=JsonSchemaEncoder, ensure_ascii=False, indent=2)


def load_usage(evaluated_model: GPTModel, evaluated_operation_name: str) -> GPTUsage:
    """
    Loads a stored model usage of an evaluated operation.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation
    :return: GPT model usage
    """
    file_name = get_usage_file_name(evaluated_model, evaluated_operation_name)
    with open(file_name, 'r', encoding='utf-8') as f:
        return GPTUsage.from_dict(json.load(f))

## Prepare Testbed
The testbed is prepared.
For the generation of random lists of single-digit integers, the seed of the PRNG is set to `0`.
To evaluate the performance of the language model, an evaluator function is created for each cardinality to evaluate.

In [20]:
seed = 0

In [21]:
from typing import Callable
from pure_graph_of_thoughts.api.state import State
from auto_graph_of_thoughts.baseline.model import BaselineIterationResult
from pure_graph_of_thoughts.api.graph.operation import GraphOfOperations


def create_graph_of_operations_evaluator(
        controller: CompleteGraphController,
        init_state_generator: Callable[[], State],
        validate_output: Callable[[State, State], bool]
) -> Callable[[GraphOfOperations, int], BaselineIterationResult]:
    """
    Creates a graph of operations evaluator function based on an initial state generator.
    :param controller: controller
    :param init_state_generator: initial state generator
    :param validate_output: output validator
    :return: graph of operations evaluator function
    """

    def evaluate_graph_of_operations(graph_of_operations: GraphOfOperations, iteration: int) -> BaselineIterationResult:
        """
        Evaluates a graph of operations.
        :param graph_of_operations: graph of operations to evaluate
        :param iteration: current iteration
        :return: baseline iteration result
        """
        init_state: State = init_state_generator()
        graph_of_thoughts = controller.execute_graph(
                graph_of_operations=graph_of_operations,
                init_state=init_state
        )
        out_states = [sink.thought.state for sink in graph_of_thoughts.sinks]
        is_valid = validate_output(init_state, out_states)
        return BaselineIterationResult(
                graph_of_operations=graph_of_operations.to_schema(),
                graph_of_thoughts=graph_of_thoughts.to_schema(),
                is_valid=is_valid,
                cost=len(graph_of_thoughts.nodes),
                iteration=iteration
        )

    return evaluate_graph_of_operations

In [22]:
from typing import Sequence
from pure_graph_of_thoughts.api.operation import Operation
from auto_graph_of_thoughts.baseline.input_output_baseline_strategy import InputOutputBaselineStrategy
from auto_graph_of_thoughts.baseline.model import BaselineResultSummary
from random import Random


def create_single_list_init_state_generator(list_cardinality: int) -> Callable[[], State]:
    """
    Creates an initial state generator.
    :param list_cardinality: list cardinality
    :return: init state generator
    """
    rnd = Random(seed)
    return lambda: {
        'list': [
            rnd.randint(0, 9) for _ in range(list_cardinality)
        ]
    }


def create_sublist_init_state_generator(list_cardinality: int) -> Callable[[], State]:
    """
    Creates an initial state generator for generating two sublists.
    :param list_cardinality: total list cardinality
    :return: init state generator
    """
    sublist_cardinality_a = list_cardinality // 2
    sublist_cardinality_b = list_cardinality - sublist_cardinality_a
    rnd = Random(seed)
    return lambda: {
        'lists': [
            [
                rnd.randint(0, 9) for _ in range(sublist_cardinality_a)
            ],
            [
                rnd.randint(0, 9) for _ in range(sublist_cardinality_b)
            ]
        ]
    }


def create_io_baseline_strategy(
        controller: CompleteGraphController,
        operation: Operation,
        init_state_generator: Callable[[], State],
        validate_output: Callable[[State, Sequence[State]], bool]
) -> InputOutputBaselineStrategy:
    """
    Creates an IO baseline strategy for a given cardinality.
    :param controller: controller
    :param operation: operation
    :param init_state_generator: initial state generator
    :param validate_output: output state validator
    :return: IO baseline strategy
    """
    return InputOutputBaselineStrategy(
            operation=operation,
            evaluate_graph=create_graph_of_operations_evaluator(controller, init_state_generator, validate_output)
    )

## Run Evaluations
The evaluation is performed for the cardinalities `8` to `64` (inclusive) with `100` results each.

In [23]:
cardinalities = list(range(8, 64 + 1))

## Run Evaluation of `op_split`
The operation `op_split` is evaluated.

In [24]:
from auto_graph_of_thoughts.tasks.sum_list import op_split, validate_op_split

op_split_name = 'op_split'
validate_output_op_split = lambda init_state, out_states: validate_op_split(
    init_state,
    out_states[0],
    out_states
)

def evaluate_op_split(evaluated_model: GPTModel) -> None:
    """
    Evaluates the operation op_split on the given model.
    :param evaluated_model: evaluated model
    """
    chat_gpt = create_chat_gpt(evaluated_model)
    ctrl = CompleteGraphController(language_model=chat_gpt)
    for cardinality in cardinalities:
        io_baseline_strategy = create_io_baseline_strategy(
                ctrl,
                op_split,
                create_single_list_init_state_generator(cardinality),
                validate_output_op_split
        )
        baseline_result_summary = io_baseline_strategy.generate(100)
        store_result(evaluated_model, op_split_name, cardinality, baseline_result_summary)
        store_usage(evaluated_model, op_split_name, chat_gpt.usage)

## Run Evaluation of `op_split` on `gpt-3.5-turbo-0125`
The operation `op_split` is evaluated on `gpt-3.5-turbo-0125`.

In [25]:
#evaluate_op_split(GPTModel.GPT_35_TURBO_0125)

## Load Result Data
The result data is loaded from disk for further analysis.

In [26]:
from typing import Dict

def load_evaluation_results(evaluated_model: GPTModel, evaluated_operation_name: str) -> Dict[int, BaselineResultSummary]:
    """
    Loads evaluation results from JSON files.
    :param evaluated_model: evaluated model
    :param evaluated_operation_name: evaluated operation name
    :return: evaluation results
    """
    results: Dict[int, BaselineResultSummary] = {}
    for evaluated_cardinality in cardinalities:
        results[evaluated_cardinality] = load_result(evaluated_model, evaluated_operation_name, evaluated_cardinality)
    return results

In [27]:
gpt35turbo0125_op_split_results = load_evaluation_results(GPTModel.GPT_35_TURBO_0125, op_split_name)

## Visualize Results
The results of the evaluation are visualized.

In [28]:
COLOR_VALID = '#6acc64'
COLOR_INVALID = '#d65f5f'

### Bar Chart All Results (atomic)

In [29]:
import plotly.express as px
import pandas as pd

is_valid = 'valid'
is_invalid = 'invalid'
validity_map = {True: is_valid, False: is_invalid}

def visualize_results_atomic(evaluated_model: GPTModel, evaluation_results: Dict[int, BaselineResultSummary]) -> None:
    """
    Visualizes evaluation results.
    :param evaluated_model: evaluated model
    :param evaluation_results: evaluation results
    """

    df_results_atomic = pd.DataFrame({
        'cardinality': [cardinality for (cardinality, summary) in evaluation_results.items() for _ in summary.results],
        'validity': [validity_map[result.is_valid] for (cardinality, summary) in evaluation_results.items() for result in
                     summary.results],
        'n_results': [1 for summary in evaluation_results.values() for _ in summary.results]
    })
    fig = px.bar(
            df_results_atomic,
            x='cardinality',
            y='n_results',
            color='validity',
            title=f'Evaluation Results for {evaluated_model.id}',
            color_discrete_map={is_valid: COLOR_VALID, is_invalid: COLOR_INVALID},
            template='simple_white',
            labels={
                'cardinality': 'list cardinality',
                'n_results': 'number of results'
            },
            height=400
    )
    fig.update_xaxes(dtick=1)
    fig.show()

In [30]:
visualize_results_atomic(GPTModel.GPT_35_TURBO_0125, gpt35turbo0125_op_split_results)

In [31]:
#fig.write_image(f'diagrams/chatgpt_capability_evaluation_op_split.jpeg', scale=10)

## Determine Success Probabilities
The probability of a successful operation for each list cardinality is determined.

In [32]:
gpt35turbo0125_probabilities_op_split = [
    {
        'cardinality': evaluated_cardinality,
        'probability': len([result.is_valid for result in summary.results if result.is_valid]) / 100.0
    }
    for (evaluated_cardinality, summary) in gpt35turbo0125_op_split_results.items()
]
gpt35turbo0125_probabilities_op_split

[{'cardinality': 8, 'probability': 1.0},
 {'cardinality': 9, 'probability': 1.0},
 {'cardinality': 10, 'probability': 1.0},
 {'cardinality': 11, 'probability': 1.0},
 {'cardinality': 12, 'probability': 0.98},
 {'cardinality': 13, 'probability': 0.99},
 {'cardinality': 14, 'probability': 0.9},
 {'cardinality': 15, 'probability': 0.87},
 {'cardinality': 16, 'probability': 1.0},
 {'cardinality': 17, 'probability': 0.97},
 {'cardinality': 18, 'probability': 0.97},
 {'cardinality': 19, 'probability': 0.99},
 {'cardinality': 20, 'probability': 0.99},
 {'cardinality': 21, 'probability': 1.0},
 {'cardinality': 22, 'probability': 0.96},
 {'cardinality': 23, 'probability': 0.93},
 {'cardinality': 24, 'probability': 0.97},
 {'cardinality': 25, 'probability': 0.83},
 {'cardinality': 26, 'probability': 0.83},
 {'cardinality': 27, 'probability': 0.79},
 {'cardinality': 28, 'probability': 0.53},
 {'cardinality': 29, 'probability': 0.75},
 {'cardinality': 30, 'probability': 0.65},
 {'cardinality': 31,